# **Neural Machine Translation**

We build a neural machine translation model based on the sequence-to-sequence (seq2seq) models proposed by Sutskever et al., 2014 and Cho et al., 2014. The seq2seq model is widely used in Machine Translation systems such as Google’s neural machine translation system (GNMT) (Wu et al., 2016).

3 files are required for this project:
1. **data.30.vi** - a file where each line contains a Vietnamese sentence to be translated (i.e. the source sentences)
2. **data.30.en** - a file where each line contains an English sentence corresponding to the Vietnamese sentence in the same line position. (i.e. the target sentences)
3. **nmt_model_keras.py** - code for this project.


##**LanguageDict**

LanguageDict is a class for creating language dict objects.

## **The load_dataset() Method**

This helper method reads from the source and target files to load max_num_examples sentences, split those sentences into train, development and test sets, and return relevant data.

As an example of the ouput returned by this code, let's assume we are translating the sentence 'I like rabbits' from English to English (this of course is never the case), such that the tokenised and case-normalised source sentence list and target sentence list are as follows:


```
# In Vietnamese this would actually be [['tôi', 'thích', 'thỏ']].
# We will use English to English here using the following code.
source_words = [['i', 'like', 'rabbits']]
target_words = [['i', 'like', 'rabbits']]
```
The word2ids for the source and target language dictionaries look as follows:
```
source_dict.word2ids = {'<PAD>': 0, '<UNK>': 1, 'i': 2, 'like': 3, 'rabbits':4}

# end and start tokens are added to the target words
target_dict.word2ids = {'<PAD>': 0, '<UNK>': 1, '<start>': 2, 'i': 3, 'like': 4, 'rabbits':5, '<end>':6}

```
Let's also assume that we are training and testing on this dataset of one sentence.
The **source words** for train/dev/test will be given as follows:
```
# [batch_size X max_sent_length]
source_words_train = [[2,3,4]] # corresponding to ['i', 'like', 'rabbits']
source_words_dev = [[2,3,4]]  # corresponding to ['i', 'like', 'rabbits']
source_words_test = [[2,3,4]] # corresponding to ['i', 'like', 'rabbits']
```

The **target words** for the train data will be given as follows (dev/test do not need target words as the model will generate those):
```
target_words_train = [[2,3,4,5]] # corresponding to ['<start>', 'i', 'like', 'rabbits']
```

The **target words labels** for each word will be the next word. The target word labels for train/dev/test data will be given as follows
```
target_words_train_labels = [[3,4,5,6]] # corresponding to ['i', 'like', 'rabbits', '<end>']
target_words_dev_labels = [[3,4,5,6]] # corresponding to ['i', 'like', 'rabbits', '<end>']
target_words_test_labels = [[3,4,5,6]] # corresponding to ['i', 'like', 'rabbits', '<end>']
```
The dimensions for the train target words labels would be expanded to have the following dimentionality:
```
# [batch_size X max_sent_length array X 1]
[[3], [4], [5], [6]]
```






##**Neural Translation Model (NMT)**

For NMT, the network (a system of connected layers/models) used for training differs slightly from the network used for inference. Both use the encoder-decoder architecture.




Summary:

This code snippet outlines the implementation of a Neural Machine Translation (NMT) model, which consists of an encoder-decoder architecture with attention mechanism. The tasks are divided into three parts:
1. Task 1: Implementing the Encoder
   - Embedding Lookup: Two embedding layers are created for the source and target languages to convert input word indices into dense vectors.
   - Encoder LSTM: An LSTM layer is utilized to process the embedded source words, producing encoder outputs and final states.

2. Task 2: Implementing the Decoder and the Inference Loop
   - Get Decoder Outputs: The decoder LSTM is employed to generate outputs during inference using the initial states from the encoder. The outputs are optionally passed through an attention layer and a dense layer to obtain the final predicted outputs.

3. Task 3: Adding Attention
   - Attention Calculation: Attention scores are computed using the dot product between decoder and encoder outputs, followed by the softmax function to obtain attention weights.
   - Context Vector: A context vector is generated by taking the weighted sum of encoder outputs based on attention weights.
   - Concatenation with Decoder Outputs: The context vector is concatenated with the decoder outputs to incorporate context information during decoding.

The model's performance is evaluated on a development set and a test set, yielding a BLEU score of 8.04 on the test set. The training time for the model is approximately 45 minutes and 53 seconds. Additionally, warnings regarding tokenization issues are addressed to ensure accurate evaluation. Overall, the NMT model demonstrates effective translation capabilities with attention mechanism integration.

###**Training mode**

**Encoder**

Given:
- `source_words`: a `batch_size(num_sents) x max_sentence_length` array representing the source words. In our mini example, this would be the Vietnamese equivalent of `['i', 'like', 'rabbits']`, i.e. `[['tôi', 'thích', 'thỏ']]`.

The following steps comprise the encoder network:

1. Transform `source_words` into `source_words_embeddings` using a randomly initialized embedding lookup. `source_words_embeddings` is thus an array with the shape `batch_size(num_sents) x max_sentence_length x embedding_dim`.
2. Apply embedding dropout with `embedding_dropout_rate`.
3. Use a single `LSTM` with the `hidden_size` units to learn a representation for the source words i.e. to encode the input.

    (a.) The hidden and cell states for this `LSTM` are initialized to zeros (i.e. we leave the `initial_state = None` default as is).

    (b.) We save the `encoder_outputs` (the sequence not just the last state); and the encoder (hidden and cell) states.

This way, the model encodes a representation for the source words. Task 1 guides you to complete the encoder part of the training model.


**Decoder (No Attention)**

Given:
- `target_words`: a `batch_size(i.e. num_sents in batch) x max_sentence_length` array representing the target words. This is a time shifted translation of the source words with an added (prepended) `<START>` token `['<start>', 'i', 'like', 'rabbits']`.

The decoding is done in the following steps:

1. Transform `target_words` into `target_words_embeddings` using a randomly initialized embedding lookup. `target_words_embeddings` is thus an array with the shape `batch_size x max_sentence_length x embedding_dim`.

2. Apply embedding dropout of `embedding_dropout_rate`.

3. Use a single `LSTM` with `hidden_size` units to learn a representation for the target words. The context is given to this model by using the encoder states to initialise the decoder LSTM. For example, the encoder state for `'thỏ'` (last word in the input sequence, its hidden representation summarises the sentence) is used to learn the representation for the `'<start>'` token.

4. For each token representation, we use a dense layer to output a `target_vocab_size` vector of probabilities to be the next word following the represented token. The output `decoder_outputs_train` is thus an array  with the shape `batch_size x max_sent_length + 1 x target_vocab_size`.


###**Inference Mode**

**Encoder**

The inference time encoding follows the same steps as the training time encoding.


**Decoder (No attention)**

During training time, we passed a `batch_size(num_sents) x max_sentence_length` array representing the target words into the decoder LSTM. The `decoder_lstm` represents the given target sentence using the context from the encoder LSTM (representation for the source sentence).  

At test time, several things are different:

1. We no longer have access to a complete translation of the source sentence (recall that no `target_words` arrays exist for dev and test sets). Rather we initialise the target words array as follows:

    Each expected target sentence contains only a single token index, the index of the `'<start>'` token. So, the target_word_dev/test is a `batch_size x 1` array (see the nmt.eval() function).

2. This `batch_size x 1` array is fed to the trained `decoder_lstm` and the predicted array is a `batch_size x 1 x target_vocab_size` such that taking the argmax of this array across the dimension 2 will give the most probable next word.

For example, at time_step 0 (first time step) the `step_target_words` is given. It is a `batch_size x 1` array containing the `'<start>'` token. The next word prediction of the decoder is for each sentence (in the batch) the first actual word.


At the first time step, the `decoder_lstm` still uses the `encoder_states` as its initial states. At subsequent time steps, it uses its own states from the previous time steps. We hence loop over time steps to generate a new word at a time.





In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# change this to the path to your folder. Remember to start from the home directory
PATH = 'MyDrive/Neural_Networks_Activity_1'

In [3]:
PATH_TO_FOLDER = "/content/drive/" + PATH

In [4]:
import sys
sys.path.append(PATH_TO_FOLDER)

In [5]:
SOURCE_PATH = PATH_TO_FOLDER + '/data.30.vi'
TARGET_PATH = PATH_TO_FOLDER + '/data.30.en'

Let's install the Sacrebleu (https://github.com/mjpost/sacrebleu) package for BLEU computation.

In [6]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.4 MB/s eta 0:00:00


In [7]:
import nmt_model_keras as nmt

In [8]:
nmt.main(SOURCE_PATH, TARGET_PATH, use_attention=False)

loading dictionaries
read 24000/3000/3000 train/dev/test batches
number of tokens in source: 2034, number of tokens in target:2506
Task 1(a): Creating the embedding lookups...

Task 1(b): Looking up source and target words...

Task 1(c): Creating an encoder


						 Train Model Summary.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 100)            203400    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 100)            250600    ['input_2[0][0]']             
                                                                  

Model BLEU score: 0.30
Time used for evaluate on dev set: 0 m 13 s
Starting training epoch 2/10
240/240 [==============================] - 19s 79ms/step - loss: 4.4836 - accuracy: 0.2359
Time used for epoch 2: 0 m 19 s
Evaluating on dev set after epoch 2/10:
30/30 [==============================] - 0s 2ms/step


Model BLEU score: 1.57
Time used for evaluate on dev set: 0 m 12 s
Starting training epoch 3/10
240/240 [==============================] - 17s 69ms/step - loss: 4.2009 - accuracy: 0.2694
Time used for epoch 3: 0 m 16 s
Evaluating on dev set after epoch 3/10:
30/30 [==============================] - 0s 2ms/step


Model BLEU score: 1.27
Time used for evaluate on dev set: 0 m 11 s
Starting training epoch 4/10
240/240 [==============================] - 16s 67ms/step - loss: 4.0309 - accuracy: 0.2876
Time used for epoch 4: 0 m 16 s
Evaluating on dev set after epoch 4/10:
30/30 [==============================] - 0s 3ms/step


...

Model BLEU score: 3.51
Time used for evaluate on dev set: 0 m 11 s
Starting training epoch 8/10
240/240 [==============================] - 15s 62ms/step - loss: 3.5855 - accuracy: 0.3471
Time used for epoch 8: 0 m 15 s
Evaluating on dev set after epoch 8/10:
30/30 [==============================] - 0s 2ms/step


Model BLEU score: 3.79
Time used for evaluate on dev set: 0 m 12 s
Starting training epoch 9/10
240/240 [==============================] - 15s 64ms/step - loss: 3.5083 - accuracy: 0.3573
Time used for epoch 9: 0 m 15 s
Evaluating on dev set after epoch 9/10:
30/30 [==============================] - 0s 2ms/step


Model BLEU score: 4.43
Time used for evaluate on dev set: 0 m 12 s
Starting training epoch 10/10
240/240 [==============================] - 15s 64ms/step - loss: 3.4402 - accuracy: 0.3646
Time used for epoch 10: 0 m 20 s
Evaluating on dev set after epoch 10/10:
30/30 [==============================] - 0s 3ms/step


Model BLEU score: 4.58
Time used for evaluate on dev set: 0 m 12 s
Training finished!
Time used for training: 6 m 7 s
Evaluating on test set:
30/30 [==============================] - 0s 5ms/step


Model BLEU score: 5.13
Time used for evaluate on test set: 0 m 12 s


##**Training and Decoding with Attention**

The inputs to the attention layer are encoder and decoder outputs. The attention mechanism:
1. Computes a score (Luong's dot product attention score) for each source word
2. Weights the encoder representations using these scores.
3. Concatenates the weighted encoder representation with the decoder ouput.
This new decoder output will now be the input to the `decoder_dense` layer.

Step-by-step details for Task 3 are in the pdf file. Once you have completed this Task, you are ready to train with attention. Training time will be no more than 10 minutes using a GPU and you should get a test BLEU score of around 10.

In [9]:
from keras.layers import Concatenate
from keras.layers import Dot
nmt.main(SOURCE_PATH, TARGET_PATH, use_attention=True)

loading dictionaries
read 24000/3000/3000 train/dev/test batches
number of tokens in source: 2034, number of tokens in target:2506
Task 1(a): Creating the embedding lookups...

Task 1(b): Looking up source and target words...

Task 1(c): Creating an encoder


						 Train Model Summary.
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 100)            203400    ['input_6[0][0]']             
                                                                                                  
 input_7 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 dropout_2 (Dropout)         (None, None, 100)            0         ['embedding_2[0][0]']         
                                                                

Model BLEU score: 1.44
Time used for evaluate on dev set: 0 m 14 s
Starting training epoch 2/10
240/240 [==============================] - 19s 80ms/step - loss: 4.3401 - accuracy: 0.2556
Time used for epoch 2: 0 m 20 s
Evaluating on dev set after epoch 2/10:
30/30 [==============================] - 0s 3ms/step


Model BLEU score: 1.64
Time used for evaluate on dev set: 0 m 13 s
Starting training epoch 3/10
240/240 [==============================] - 16s 68ms/step - loss: 4.0594 - accuracy: 0.2889
Time used for epoch 3: 0 m 16 s
Evaluating on dev set after epoch 3/10:
30/30 [==============================] - 0s 5ms/step


Model BLEU score: 2.48
Time used for evaluate on dev set: 0 m 13 s
Starting training epoch 4/10
240/240 [==============================] - 16s 67ms/step - loss: 3.8594 - accuracy: 0.3128
Time used for epoch 4: 0 m 20 s
Evaluating on dev set after epoch 4/10:
30/30 [==============================] - 0s 3ms/step


...

Model BLEU score: 5.90
Time used for evaluate on dev set: 0 m 13 s
Starting training epoch 8/10
240/240 [==============================] - 16s 69ms/step - loss: 3.3131 - accuracy: 0.3828
Time used for epoch 8: 0 m 20 s
Evaluating on dev set after epoch 8/10:
30/30 [==============================] - 0s 3ms/step


Model BLEU score: 6.66
Time used for evaluate on dev set: 0 m 12 s
Starting training epoch 9/10
240/240 [==============================] - 17s 69ms/step - loss: 3.1864 - accuracy: 0.3991
Time used for epoch 9: 0 m 20 s
Evaluating on dev set after epoch 9/10:
30/30 [==============================] - 0s 3ms/step


Model BLEU score: 7.51
Time used for evaluate on dev set: 0 m 13 s
Starting training epoch 10/10
240/240 [==============================] - 15s 64ms/step - loss: 3.0642 - accuracy: 0.4142
Time used for epoch 10: 0 m 15 s
Evaluating on dev set after epoch 10/10:
30/30 [==============================] - 0s 3ms/step


Model BLEU score: 8.18
Time used for evaluate on dev set: 0 m 13 s
Training finished!
Time used for training: 5 m 38 s
Evaluating on test set:
30/30 [==============================] - 0s 3ms/step


Model BLEU score: 8.27
Time used for evaluate on test set: 0 m 12 s


1. **Model Architecture**: An LSTM-based encoder-decoder architecture is used with attention mechanisms. This is a common setup for sequence-to-sequence tasks like translation.

2. **Training Progress**: The training progresses through epochs, with each epoch updating the model's parameters to minimize the loss function. Each epoch takes a certain amount of time to complete, which seems reasonable given the complexity of the model and the size of the dataset.

3. **Evaluation on Dev Set**: After each epoch, the model's performance is evaluated on a development set. This evaluation likely involves calculating metrics such as BLEU score to assess translation quality.

4. **BLEU Scores**: The BLEU scores indicate how well the model is performing on the development set. As the training progresses, the BLEU scores are increasing, which suggests that the model is improving its translation quality over time.

5. **Model BLEU Score**: The reported BLEU scores for the model are between 1.44 and 8.27, which indicates the quality of translations produced by the model. A higher BLEU score generally indicates better translation quality.

Overall, it seems like the model is learning effectively and improving its translation quality over the training epochs.